In [0]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import matplotlib.pylab as plt
import numpy as np
import PIL.Image as Image

IMAGE_SHAPE = (96, 96)


tf.keras.utils.get_file(
  'training','https://data.deic.dk/public.php?service=files&t=2fd47962a38e2a70570f3be027cea57f&download',
   untar=True)
tf.keras.utils.get_file(
  'test','https://data.deic.dk/public.php?service=files&t=53f154ca9e9f1e6aee8587f5d18f81fd&download',
   untar=True)

1381154816/1381151715 [==============================] - 134s 0us/step


'/root/.keras/datasets/test'

In [0]:
train_annotation_url = 'https://data.deic.dk/public.php?service=files&t=8dc110f312677d2b53003de983b3a26e&download'
test_annotation_url = 'https://data.deic.dk/public.php?service=files&t=c899715d20e2e80063ced63d9cfec9c3&download'

tf.keras.utils.get_file('annotation', train_annotation_url, untar=True)
tf.keras.utils.get_file('test.json', test_annotation_url, untar=True)

98304/91660 [================================] - 1s 14us/step


'/root/.keras/datasets/test.json'

In [0]:
import json

def get_na_fungi_filenames(annotations, fungi_set, is_train=True, image_ids=set()):
  if (is_train):
    with open(annotations) as JSON:
      data = json.load(JSON)
      category_id_set = set()
      for mushroom in data['categories']:
          if mushroom['name'].split('var.')[0] in fungi_set:
              category_id_set.add(mushroom['id'])
      image_id_set = set()
      for annotation in data['annotations']:
          if annotation['category_id'] in category_id_set:
              image_id_set.add(annotation['image_id'])
      na_fungi = set()
      for image in data['images']:
          if image['id'] in image_id_set:
              na_fungi.add(image['file_name'])
      return category_id_set, na_fungi
  else:
      with open(annotations) as JSON:
        data = json.load(JSON)
        na_fungi = set()
        for image in data['images']:
          if image['id'] in image_ids:
            na_fungi.add(image['file_name'])
        return None, na_fungi

# Select NA fungi and place their names in train_set, val_set, and test_set
with open(r'/content/fungi_us.txt') as file:
    fungi_data = file.readlines()
fungi_set = set()
for fungi in fungi_data:
    fungi_set.add(fungi.strip())

id_set, train_set = get_na_fungi_filenames(r'/root/.keras/datasets/train.json', fungi_set)
_, val_set = get_na_fungi_filenames(r'/root/.keras/datasets/val.json', fungi_set)
_, test_set = get_na_fungi_filenames(r'/root/.keras/datasets/test.json', fungi_set, False, id_set)

print(len(train_set))
print(len(val_set))

27022
969


In [0]:
import shutil
import os

for image in train_set:
  dest_fpath = '/root/.keras/datasets/filtered/' + image
  os.makedirs(os.path.dirname(dest_fpath), exist_ok=True)
  shutil.copy('/root/.keras/datasets/' + image, dest_fpath)

for image in val_set:
  dest_fpath = '/root/.keras/datasets/filtered/' + image
  os.makedirs(os.path.dirname(dest_fpath), exist_ok=True)
  shutil.copy('/root/.keras/datasets/' + image, dest_fpath)

In [0]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())


16384/10484 [==============================================] - 0s 0us/step


In [0]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    validation_split=0.2,
    )
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [0]:
train_generator = train_datagen.flow_from_directory(
    '/root/.keras/datasets/filtered/images',
    target_size=IMAGE_SHAPE,
    subset="training",
    batch_size=32)
validation_generator = train_datagen.flow_from_directory(
    '/root/.keras/datasets/filtered/images',
    target_size=IMAGE_SHAPE,
    subset="validation")

for image_batch, label_batch in train_generator:
  print("Image batch shape: ", image_batch.shape)
  print("Label batch shape: ", label_batch.shape)
  break

Found 22514 images belonging to 323 classes.
Found 5477 images belonging to 323 classes.
Image batch shape:  (32, 96, 96, 3)
Label batch shape:  (32, 323)


In [0]:
feature_extractor_url = "https://tfhub.dev/google/imagenet/mobilenet_v2_050_96/feature_vector/4" #@param {type:"string"}
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=(96,96,3))

feature_batch = feature_extractor_layer(image_batch)
feature_extractor_layer.trainable = False

model = tf.keras.Sequential([
  feature_extractor_layer,
  layers.Dense(train_generator.num_classes, activation='softmax')
])

model.summary()

predictions = model(image_batch)

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
  loss='categorical_crossentropy',
  metrics=['acc', 'top_k_categorical_accuracy'])

class CollectBatchStats(tf.keras.callbacks.Callback):
  def __init__(self):
    self.batch_losses = []
    self.batch_acc = []

  def on_train_batch_end(self, batch, logs=None):
    if batch % 100 == 0:
      self.batch_losses.append(logs['loss'])
      self.batch_acc.append(logs['acc'])
      self.model.reset_metrics()

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)

batch_stats_callback = CollectBatchStats()

history = model.fit_generator(train_generator, epochs=10,
                              steps_per_epoch=steps_per_epoch,
                              validation_data=validation_generator,
                              validation_freq=1,
                              callbacks = [batch_stats_callback])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              706224    
_________________________________________________________________
dense (Dense)                (None, 323)               413763    
Total params: 1,119,987
Trainable params: 413,763
Non-trainable params: 706,224
_________________________________________________________________
Epoch 1/30
162/704 [=====>........................] - ETA: 4:11 - loss: 6.6521 - acc: 0.1613 - top_k_categorical_accuracy: 0.3723

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:742: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


704/704 [==============================] - 396s 563ms/step - loss: 6.2852 - acc: 0.1458 - top_k_categorical_accuracy: 0.3646 - val_loss: 6.6485 - val_acc: 0.1614 - val_top_k_categorical_accuracy: 0.3721
Epoch 2/30
704/704 [==============================] - 362s 515ms/step - loss: 4.6486 - acc: 0.2292 - top_k_categorical_accuracy: 0.5104 - val_loss: 7.3502 - val_acc: 0.1640 - val_top_k_categorical_accuracy: 0.3706
Epoch 3/30
704/704 [==============================] - 344s 489ms/step - loss: 4.1040 - acc: 0.2500 - top_k_categorical_accuracy: 0.5833 - val_loss: 7.8760 - val_acc: 0.1696 - val_top_k_categorical_accuracy: 0.3730
Epoch 4/30
704/704 [==============================] - 351s 499ms/step - loss: 3.7749 - acc: 0.3958 - top_k_categorical_accuracy: 0.6146 - val_loss: 8.6067 - val_acc: 0.1735 - val_top_k_categorical_accuracy: 0.3867
Epoch 5/30
704/704 [==============================] - 359s 510ms/step - loss: 3.5108 - acc: 0.3750 - top_k_categorical_accuracy: 0.6146 - val_loss: 8.9479 

KeyboardInterrupt: ignored

In [0]:
plt.figure()
plt.ylabel("Loss")
plt.xlabel("Training Steps")
plt.ylim([0,10])
plt.plot(batch_stats_callback.batch_losses)

plt.figure()
plt.ylabel("Accuracy")
plt.xlabel("Training Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback.batch_acc)

class_names = sorted(train_generator.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
class_names

predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)

plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range(30):
  plt.subplot(4,3,n+1)
  plt.imshow(image_batch[n])
  color = "green" if predicted_id[n] == label_id[n] else "red"
  plt.title(predicted_label_batch[n].title(), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect)")